In [1]:
from lxml import html
import requests

In [2]:
url = 'https://presidentialopenquestions.com/'
headers = {
    'human': 'Jackie Kazil',
    'human_twitter': '@jackiekazil',
    'human_email': 'jackiekazil@gmail.com',
    'notes': 'If this causes you issues, please contact me. I am pulling data \
              to release, so that people can group similar questions, before \
              debate.'
}


request = requests.get(url, headers=headers)

In [3]:
print(request)
print(dir(request))

print(request.json)

<Response [200]>
['__attrs__', '__bool__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__iter__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__nonzero__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_content', '_content_consumed', 'apparent_encoding', 'close', 'connection', 'content', 'cookies', 'elapsed', 'encoding', 'headers', 'history', 'is_permanent_redirect', 'is_redirect', 'iter_content', 'iter_lines', 'json', 'links', 'ok', 'raise_for_status', 'raw', 'reason', 'request', 'status_code', 'text', 'url']
<bound method Response.json of <Response [200]>>


In [4]:
tree = html.fromstring(request.content)

In [5]:
import dateparser

def convert_datetime(x):
    # example: 'Oct. 2, 2016, 10:13 p.m.'
    x = dateparser.parse(x)
    x = x.strftime('%Y-%m-%dT%H:%M:%S')
    return x

In [6]:
questions = tree.xpath('//div[2]/p[1]/strong/a/text()')
q_paths = tree.xpath('//div[2]/p[1]/strong/a/@href')
count = tree.xpath('//div[1]/div[1]/span/text()')
submitters = tree.xpath('//div[3]/strong/text()')

idea_infos = [x.strip('\n').strip() for x in tree.xpath('//div[3]/text()')]
idea_dates = [x for x in idea_infos if x not in ['', 'Submitted by:', 'Issue area:']]
idea_dates = [convert_datetime(x) for x in idea_dates]

issue_areas = tree.xpath('//div[3]/a/strong/text()')
issue_paths = tree.xpath('//div[3]/a/@href')

In [14]:
import json

data = []
zip_data = zip(questions, q_paths, count, submitters, idea_dates, issue_areas, issue_paths)
for a, b, c, d, e, f, g in zip_data:
    submit_info = [x.strip() for x in d.split('from')]
    
    submitter = submit_info[0]
    if len(submit_info) > 1: 
        location = submit_info[1]
    else: 
        location = None
    
    record = {
        'question': a,
        'question_path': b,
        'vote_count': c, 
        'submitters': submit_info[0],
        'submitter_location': submit_info[1],
        'submission_date': e,
        'issue_area': f, 
        'issue_path': g,
    }
    data.append(record)

data[0]

{'issue_area': 'Civil Rights',
 'issue_path': '/category/3/',
 'question': 'What would you do to protect disabled people from abusive treatments?',
 'question_path': '/questions/11494/vote/',
 'submission_date': '2016-10-03T22:09:00',
 'submitter_location': 'NY',
 'submitters': 'Eirene A.',
 'vote_count': '18'}

In [15]:
import json
#with open('data.txt', 'w') as outfile:
    #json.dump(data, outfile)
    
import pprint
pprint.pprint(data)

[{'issue_area': 'Civil Rights',
  'issue_path': '/category/3/',
  'question': 'What would you do to protect disabled people from abusive '
              'treatments?',
  'question_path': '/questions/11494/vote/',
  'submission_date': '2016-10-03T22:09:00',
  'submitter_location': 'NY',
  'submitters': 'Eirene A.',
  'vote_count': '18'},
 {'issue_area': 'Society & Community',
  'issue_path': '/category/2/',
  'question': 'Would you take any steps to combat online abuse/internet '
              'trolling?',
  'question_path': '/questions/8222/vote/',
  'submission_date': '2016-09-30T00:46:00',
  'submitter_location': 'CA',
  'submitters': 'diana p.',
  'vote_count': '18'},
 {'issue_area': 'Government & Elections',
  'issue_path': '/category/10/',
  'question': 'Do you support a direct democracy system?',
  'question_path': '/questions/17012/vote/',
  'submission_date': '2016-10-07T21:34:00',
  'submitter_location': 'WA',
  'submitters': 'Joshua M.',
  'vote_count': '9'},
 {'issue_area': 